# TailCurve vs TailBondy: Handling Challenging Development Data

This notebook demonstrates the key differences between `TailCurve` and `TailBondy` methods when dealing with challenging development data that includes:
- Negative development values
- Sparse development patterns
- LDFs close to 1.0

**Key Finding**: TailBondy is more robust for irregular development patterns.

In [1]:
import numpy as np
import pandas as pd
import chainladder as cl
import warnings
warnings.filterwarnings('ignore')

## 1. Create Challenging Development Data

We'll modify the GenIns sample data to simulate challenging development patterns.

In [37]:
# Start with GenIns sample data and modify it to create challenging patterns
genins = cl.load_sample('genins')
print(f"Original GenIns shape: {genins.shape}")

# Create challenging version by introducing negative development
triangle_mod = genins.copy()
incremental = triangle_mod.cum_to_incr()

# Introduce negative development values (simulating challenging data)
incremental.values[0,0,2,5] = -500000  # Large negative development#
incremental.values[0,0,1,5] = -500000 *2  # Large negative development
incremental.values[0,0,3,5] = -500000 *3
incremental.values[0,0,1,7] = -25000   # Another negative value
incremental.values[0,0,3,4] = -15000   # Third negative value
incremental.values[0,0,6,2] = -1_000_000   # Fourth negative value
incremental.values[0,0,4,2] = -1_000_000   # Fourth negative value
# incremental.values[0,0,:,0] = np.nan
# incremental.values[0,0,:,1] = np.nan
# Convert back to cumulative
triangle_challenging = incremental.incr_to_cum()

print(f"\nModified triangle with negative development:")
print(f"Shape: {triangle_challenging.shape}")
print(f"Development periods: {triangle_challenging.ddims}")

# Show the negative incremental values we introduced
print(f"\nIntroduced negative incremental values:")
print(f"Origin 3, Dev period 6: {incremental.values[0,0,2,5]:,.0f}")
print(f"Origin 2, Dev period 8: {incremental.values[0,0,1,7]:,.0f}")
print(f"Origin 4, Dev period 5: {incremental.values[0,0,3,4]:,.0f}")

cumulative = incremental.incr_to_cum()
cumulative

Original GenIns shape: (1, 1, 10, 10)

Modified triangle with negative development:
Shape: (1, 1, 10, 10)
Development periods: [ 12  24  36  48  60  72  84  96 108 120]

Introduced negative incremental values:
Origin 3, Dev period 6: -500,000
Origin 2, Dev period 8: -25,000
Origin 4, Dev period 5: -15,000


,12,24,36,48,60,72,84,96,108,120
2001,"357,848","1,124,788","1,735,330","2,218,270","2,745,596","3,319,994","3,466,336","3,606,286","3,833,515","3,901,463"
2002,"352,118","1,236,139","2,170,033","3,353,322","3,799,067","2,799,067","3,326,871","3,301,871","3,726,917",
2003,"290,507","1,292,306","2,218,525","3,235,179","3,985,995","3,485,995","3,981,987","4,262,392",,
2004,"310,608","1,418,858","2,195,047","3,757,447","3,742,447","2,242,447","2,448,733",,,
2005,"443,160","1,136,350","136,350","905,838","1,410,689","1,881,328",,,,
2006,"396,132","1,333,217","2,180,715","2,985,752","3,691,712",,,,,
2007,"440,832","1,288,463","288,463","1,351,732",,,,,,
2008,"359,480","1,421,128","2,864,498",,,,,,,
2009,"376,686","1,363,294",,,,,,,,
2010,"344,014",,,,,,,,,


## 2. Analyze Data Quality Issues

In [38]:
# Check for negative development
negative_count = np.sum(incremental.values < 0)
print(f"Negative incremental values: {negative_count}")

# Calculate LDFs from the challenging triangle
dev = cl.Development().fit_transform(triangle_challenging)
ldfs = dev.ldf_.values[~np.isnan(dev.ldf_.values)]

print(f"\nLDF Analysis:")
print(f"Total LDFs: {len(ldfs)}")
print(f"Min LDF: {np.min(ldfs):.4f}")
print(f"Max LDF: {np.max(ldfs):.4f}")
print(f"LDFs ≤ 1.0: {np.sum(ldfs <= 1.0)}")
print(f"LDFs < 1.05: {np.sum(ldfs < 1.05)} (close to 1.0, challenging for curve fitting)")

print(f"\nAll LDFs: {ldfs}")

Negative incremental values: 7

LDF Analysis:
Total LDFs: 9
Min LDF: 0.8754
Max LDF: 3.4906
LDFs ≤ 1.0: 1
LDFs < 1.05: 3 (close to 1.0, challenging for curve fitting)

All LDFs: [3.49060655 1.34510058 1.6300609  1.1774266  0.8753514  1.1161784
 1.03669122 1.09442099 1.01772473]


## 3. Test TailCurve Methods

In [39]:
print("Testing TailCurve methods:")
print("-" * 40)

curve_types = ['exponential', 'inverse_power', 'weibull']
tc_results = {}

for curve in curve_types:
    try:
        tail = cl.TailCurve(
            curve=curve, 
            errors='ignore', 
            reg_threshold=(1.001, None),
            fit_period=(24, None)  # Start fitting from 24 months
        )
        fitted = tail.fit_transform(dev.copy())
        
        tail_factor = fitted.tail_.iloc[0,0]
        
        # Check if parameters are reasonable
        if hasattr(tail, '_intercept_') and hasattr(tail, '_slope_'):
            intercept = tail._intercept_.flatten()[0] if tail._intercept_.size > 0 else np.nan
            slope = tail._slope_.flatten()[0] if tail._slope_.size > 0 else np.nan
            params_ok = np.isfinite([intercept, slope]).all() and np.abs(tail_factor) < 1e10
        else:
            intercept = slope = np.nan
            params_ok = np.abs(tail_factor) < 1e10
            
        tc_results[curve] = {
            'success': True, 
            'tail_factor': tail_factor, 
            'params_ok': params_ok,
            'intercept': intercept,
            'slope': slope
        }
        
        print(f"{curve:12s}: ✓ Tail = {tail_factor:.4f}, Reasonable = {params_ok}")
        
    except Exception as e:
        tc_results[curve] = {'success': False, 'error': str(e)}
        print(f"{curve:12s}: ✗ Failed - {str(e)[:60]}...")

tc_success_count = sum(1 for r in tc_results.values() if r['success'])
tc_reasonable_count = sum(1 for r in tc_results.values() if r.get('success') and r.get('params_ok'))
tc_success_rate = tc_success_count / len(tc_results)

print(f"\nTailCurve Results:")
print(f"  Success rate: {tc_success_count}/{len(tc_results)} ({tc_success_rate:.1%})")
print(f"  Reasonable results: {tc_reasonable_count}/{len(tc_results)} ({tc_reasonable_count/len(tc_results):.1%})")

Testing TailCurve methods:
----------------------------------------
exponential : ✓ Tail = 1.0537, Reasonable = True
inverse_power: ✓ Tail = 1.3634, Reasonable = True
weibull     : ✓ Tail = 1340834029643371703527085703168.0000, Reasonable = False

TailCurve Results:
  Success rate: 3/3 (100.0%)
  Reasonable results: 2/3 (66.7%)


## 4. Test TailBondy Method

In [40]:
print("Testing TailBondy configurations:")
print("-" * 40)

earliest_ages = [None, 24, 36, 48, 60]
tb_results = {}

for age in earliest_ages:
    age_str = f"earliest_age={age}" if age else "earliest_age=None"
    
    try:
        tail = cl.TailBondy(earliest_age=age, attachment_age=None)
        fitted = tail.fit_transform(dev.copy())
        
        tail_factor = fitted.tail_.iloc[0,0]
        bondy_exp = fitted.b_.iloc[0,0]
        earliest_ldf = fitted.earliest_ldf_.iloc[0,0]
        
        # Check if results are reasonable (tail factor between 1.0 and 10.0)
        reasonable = 1.0 <= tail_factor <= 10.0 and np.isfinite(bondy_exp)
        
        tb_results[age] = {
            'success': True, 
            'tail_factor': tail_factor, 
            'bondy_exp': bondy_exp,
            'earliest_ldf': earliest_ldf,
            'reasonable': reasonable
        }
        
        print(f"{age_str:17s}: ✓ Tail = {tail_factor:.4f}, Bondy = {bondy_exp:.4f}, Reasonable = {reasonable}")
        
    except Exception as e:
        tb_results[age] = {'success': False, 'error': str(e)}
        print(f"{age_str:17s}: ✗ Failed - {str(e)[:50]}...")

tb_success_count = sum(1 for r in tb_results.values() if r['success'])
tb_reasonable_count = sum(1 for r in tb_results.values() if r.get('success') and r.get('reasonable'))
tb_success_rate = tb_success_count / len(tb_results)

print(f"\nTailBondy Results:")
print(f"  Success rate: {tb_success_count}/{len(tb_results)} ({tb_success_rate:.1%})")
print(f"  Reasonable results: {tb_reasonable_count}/{len(tb_results)} ({tb_reasonable_count/len(tb_results):.1%})")

Testing TailBondy configurations:
----------------------------------------
earliest_age=None: ✓ Tail = 1.0177, Bondy = 0.5000, Reasonable = True
earliest_age=24  : ✓ Tail = 1.0278, Bondy = 0.6511, Reasonable = True
earliest_age=36  : ✓ Tail = 1.0000, Bondy = 0.2246, Reasonable = True
earliest_age=48  : ✓ Tail = 1.0069, Bondy = -0.6403, Reasonable = True
earliest_age=60  : ✓ Tail = 1.0035, Bondy = -0.5245, Reasonable = True

TailBondy Results:
  Success rate: 5/5 (100.0%)
  Reasonable results: 5/5 (100.0%)


## 5. Comparison & Recommendations

In [26]:
print("METHOD COMPARISON")
print("=" * 60)

print(f"\nSuccess & Reliability Comparison:")
print(f"  TailCurve:  {tc_success_rate:.1%} success, {tc_reasonable_count/len(tc_results):.1%} reasonable")
print(f"  TailBondy:  {tb_success_rate:.1%} success, {tb_reasonable_count/len(tb_results):.1%} reasonable")

# Show reasonable tail factor ranges
tc_reasonable_tails = [r['tail_factor'] for r in tc_results.values() 
                      if r.get('success') and r.get('params_ok') and 1.0 <= r['tail_factor'] <= 10.0]
tb_reasonable_tails = [r['tail_factor'] for r in tb_results.values() 
                      if r.get('success') and r.get('reasonable')]

if tc_reasonable_tails:
    print(f"\nTailCurve reasonable tail factors: {min(tc_reasonable_tails):.4f} - {max(tc_reasonable_tails):.4f}")
else:
    print(f"\nTailCurve: No reasonable tail factors")
    
if tb_reasonable_tails:
    print(f"TailBondy reasonable tail factors:  {min(tb_reasonable_tails):.4f} - {max(tb_reasonable_tails):.4f}")
else:
    print(f"TailBondy: No reasonable tail factors")

print("\n" + "=" * 60)
print("RECOMMENDATION")
print("=" * 60)

if tb_reasonable_count > tc_reasonable_count:
    print("\n✅ USE TAILBONDY for challenging data")
    print("\nReasons:")
    print("  • More robust with negative development")
    print("  • Better handles sparse/irregular patterns")
    print("  • Produces more reasonable tail factors")
    print("  • Actuarial methodology designed for real-world data")
    
elif tc_reasonable_count > tb_reasonable_count:
    print("\n✅ TailCurve performed better")
    print("   (This would be unusual for challenging data)")
else:
    print("\n⚠️  Both methods showed similar performance")
    print("   Consider using TailConstant with manual tail factor")

# Show best configuration
if tb_reasonable_tails:
    best_tail = min(tb_reasonable_tails, key=lambda x: abs(x - 1.05))  # Closest to 5% tail
    for age, result in tb_results.items():
        if result.get('success') and result.get('reasonable'):
            if abs(result['tail_factor'] - best_tail) < 0.01:
                print(f"\n  Recommended TailBondy config:")
                print(f"    earliest_age = {age}")
                print(f"    Expected tail factor: {best_tail:.4f}")
                break

METHOD COMPARISON

Success & Reliability Comparison:
  TailCurve:  100.0% success, 66.7% reasonable
  TailBondy:  100.0% success, 100.0% reasonable

TailCurve reasonable tail factors: 1.0432 - 1.2468
TailBondy reasonable tail factors:  1.0000 - 1.0177

RECOMMENDATION

✅ USE TAILBONDY for challenging data

Reasons:
  • More robust with negative development
  • Better handles sparse/irregular patterns
  • Produces more reasonable tail factors
  • Actuarial methodology designed for real-world data

  Recommended TailBondy config:
    earliest_age = None
    Expected tail factor: 1.0177


## 6. Usage Example

In [27]:
print("PRACTICAL USAGE EXAMPLE")
print("=" * 50)

# Demonstrate the recommended approach
print("\n# For challenging development data:")
print("import chainladder as cl")
print("")
print("# Your triangle with data quality issues")
print("triangle = your_challenging_triangle")
print("dev = cl.Development().fit_transform(triangle)")
print("")
print("# Try TailBondy first (more robust for irregular data)")
print("tail = cl.TailBondy(earliest_age=24, attachment_age=None)")
print("fitted = tail.fit_transform(dev)")
print("")
print("print(f'Tail factor: {fitted.tail_.iloc[0,0]:.4f}')")
print("print(f'Bondy exponent: {fitted.b_.iloc[0,0]:.4f}')")

# Actually run the example
print("\n" + "-" * 30)
print("RUNNING THE EXAMPLE:")
print("-" * 30)

try:
    # Use TailBondy on our challenging data
    tail_demo = cl.TailBondy(earliest_age=24, attachment_age=None)
    fitted_demo = tail_demo.fit_transform(dev)
    
    print(f"✅ TailBondy succeeded!")
    print(f"   Tail factor: {fitted_demo.tail_.iloc[0,0]:.4f}")
    print(f"   Bondy exponent: {fitted_demo.b_.iloc[0,0]:.4f}")
    print(f"   Earliest LDF: {fitted_demo.earliest_ldf_.iloc[0,0]:.4f}")
    
except Exception as e:
    print(f"❌ TailBondy failed: {e}")
    print("   Consider using TailConstant as fallback")

PRACTICAL USAGE EXAMPLE

# For challenging development data:
import chainladder as cl

# Your triangle with data quality issues
triangle = your_challenging_triangle
dev = cl.Development().fit_transform(triangle)

# Try TailBondy first (more robust for irregular data)
tail = cl.TailBondy(earliest_age=24, attachment_age=None)
fitted = tail.fit_transform(dev)

print(f'Tail factor: {fitted.tail_.iloc[0,0]:.4f}')
print(f'Bondy exponent: {fitted.b_.iloc[0,0]:.4f}')

------------------------------
RUNNING THE EXAMPLE:
------------------------------
✅ TailBondy succeeded!
   Tail factor: 1.0045
   Bondy exponent: 0.5015
   Earliest LDF: 1.7914


## Key Differences Summary

| Aspect | TailCurve | TailBondy |
|--------|-----------|----------|
| **Data Requirements** | Needs clean LDFs > 1.0 | More tolerant of irregularities |
| **Negative Development** | Can fail mathematically | Handles better |
| **Sparse Data** | Sensitive to data quality | More robust |
| **Mathematical Basis** | Parametric curve fitting | Bondy exponential decay model |
| **Parameter Control** | fit_period, curve type | earliest_age, attachment_age |
| **Best Use Case** | Clean, complete data | Challenging, irregular data |
| **Output Stability** | Can produce extreme values | Generally more stable |

### Conclusion

For development triangles with **data quality issues** such as:
- Negative development values
- Sparse or irregular patterns  
- LDFs close to 1.0
- Limited development periods

**TailBondy is the preferred method** due to its robustness and tolerance of real-world data irregularities. It's specifically designed for challenging actuarial data scenarios where traditional curve fitting methods may fail.